In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
holiday_data = pd.read_csv("holidays_events.csv")
oil_data = pd.read_csv("oil.csv")
stores_data = pd.read_csv("stores.csv")
transactions_data = pd.read_csv("transactions.csv")

In [3]:
train_data

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [4]:
print(train_data.isnull().sum())

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64


In [7]:
print(test_data.isnull().sum())

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64


In [5]:
holiday_data

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [6]:
holiday_data["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [7]:
print(holiday_data.isnull().sum())

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64


In [8]:
oil_data

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [11]:
print(oil_data.isnull().sum())

date           0
dcoilwtico    43
dtype: int64


In [14]:
print(stores_data.isnull().sum())

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64


In [15]:
transactions_data

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [16]:
print(transactions_data.isnull().sum())

date            0
store_nbr       0
transactions    0
dtype: int64


### Merging all other dataframes into train and test data frames

In [9]:
train_data = pd.merge(train_data, stores_data, on='store_nbr', how='left')
train_data = pd.merge(train_data, transactions_data, on=['date', 'store_nbr'], how='left')
train_data = pd.merge(train_data, oil_data, on='date', how='left')
train_data['dcoilwtico'] = train_data['dcoilwtico'].ffill()
train_data = pd.merge(train_data, holiday_data, on='date', how='left')

In [10]:
train_data.head(10)

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
6,6,2013-01-01,1,CELEBRATION,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
7,7,2013-01-01,1,CLEANING,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
8,8,2013-01-01,1,DAIRY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
9,9,2013-01-01,1,DELI,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [11]:
test_data = pd.merge(test_data, stores_data, on='store_nbr', how='left')
test_data = pd.merge(test_data, transactions_data, on=['date', 'store_nbr'], how='left')
test_data = pd.merge(test_data, oil_data, on='date', how='left')
test_data['dcoilwtico'] = test_data['dcoilwtico'].ffill()
test_data = pd.merge(test_data, holiday_data, on='date', how='left')

### Renaming the type columns from different data frames for clarity and removing redundant information

In [12]:
train_data.rename(columns={'type_x': 'store_type', 'type_y': 'holiday_type'}, inplace=True)
train_data.drop(columns=['locale', 'locale_name', 'description'], inplace=True)

In [13]:
test_data.rename(columns={'type_x': 'store_type', 'type_y': 'holiday_type'}, inplace=True)
test_data.drop(columns=['locale', 'locale_name', 'description'], inplace=True)

In [14]:
print(train_data.shape)
print(test_data.shape)

(3054348, 14)
(28512, 13)


## Feature Engineering

### Basic date based features 

In [15]:
train_data["date"] = pd.to_datetime(train_data["date"])
test_data["date"] = pd.to_datetime(test_data["date"])

In [16]:
train_data['year'] = train_data['date'].dt.year
train_data['dayofyear'] = train_data['date'].dt.dayofyear
train_data['quarter'] = train_data['date'].dt.quarter
train_data['is_weekend'] = (train_data['date'].dt.dayofweek >= 5).astype(int)

test_data['year'] = test_data['date'].dt.year
test_data['dayofyear'] = test_data['date'].dt.dayofyear
test_data['quarter'] = test_data['date'].dt.quarter
test_data['is_weekend'] = (test_data['date'].dt.dayofweek >= 5).astype(int)


### Cyclic features with Sin & Cos transformation

In [17]:
train_data['month_sin'] = np.sin(2 * np.pi * train_data['date'].dt.month / 12)
train_data['month_cos'] = np.cos(2 * np.pi * train_data['date'].dt.month / 12)
train_data['dayofweek_sin'] = np.sin(2 * np.pi * train_data['date'].dt.dayofweek / 7)
train_data['dayofweek_cos'] = np.cos(2 * np.pi * train_data['date'].dt.dayofweek / 7)

test_data['month_sin'] = np.sin(2 * np.pi * test_data['date'].dt.month / 12)
test_data['month_cos'] = np.cos(2 * np.pi * test_data['date'].dt.month / 12)
test_data['dayofweek_sin'] = np.sin(2 * np.pi * test_data['date'].dt.dayofweek / 7)
test_data['dayofweek_cos'] = np.cos(2 * np.pi * test_data['date'].dt.dayofweek / 7)

#### Holiday & Special Event Features

In [21]:
train_data['holiday_type'] = train_data['holiday_type'].fillna('No Holiday')
train_data['transferred'] = train_data['transferred'].fillna(False)

holiday_types = ['Holiday', 'Transfer', 'Additional', 'Bridge']
train_data['is_holiday_day'] = np.where(
    (train_data['holiday_type'].isin(holiday_types)) & (train_data['transferred'] == False), 
    1, 0
)
train_data['is_work_day'] = np.where(train_data['holiday_type'] == 'Work Day', 1, 0)
train_data.drop(columns=['holiday_type', 'transferred'], inplace=True)

In [22]:
test_data['holiday_type'] = test_data['holiday_type'].fillna('No Holiday')
test_data['transferred'] = test_data['transferred'].fillna(False)

holiday_types = ['Holiday', 'Transfer', 'Additional', 'Bridge']
test_data['is_holiday_day'] = np.where(
    (test_data['holiday_type'].isin(holiday_types)) & (test_data['transferred'] == False), 
    1, 0
)
test_data['is_work_day'] = np.where(test_data['holiday_type'] == 'Work Day', 1, 0)
test_data.drop(columns=['holiday_type', 'transferred'], inplace=True)

### Payday Feature

In [23]:
# Wages are paid on the 15th and last day of the month
train_data['is_payday'] = np.where(
    (train_data['date'].dt.day == 15) | (train_data['date'].dt.is_month_end),
    1, 0
)

test_data['is_payday'] = np.where(
    (test_data['date'].dt.day == 15) | (test_data['date'].dt.is_month_end),
    1, 0
)

### Earthquake feature

In [24]:
# Creating a flag for few week after earthquake event in 2016
earthquake_start = pd.to_datetime('2016-04-16')
earthquake_end = pd.to_datetime('2016-05-31')
train_data['post_earthquake_effect'] = np.where(
    (train_data['date'] >= earthquake_start) & (train_data['date'] <= earthquake_end),
    1, 0
)
test_data['post_earthquake_effect'] = np.where(
    (test_data['date'] >= earthquake_start) & (test_data['date'] <= earthquake_end),
    1, 0
)

In [25]:
print(train_data.shape)
print(test_data.shape)

(3054348, 24)
(28512, 23)
